# 🚀 LGS Model API V11 - PROBLEM ÇÖZÜM VERSİYONU

## ✅ Tüm Problemler İçin Çözüm:
- ✅ Temperature 0.35 (halüsinasyon önleme)
- ✅ Repetition Penalty 1.5 (döngü önleme)
- ✅ No Repeat N-gram 4 (tekrar engelleme)
- ✅ Top-K 30 (odaklı seçim)
- ✅ Sert System Prompt (format kontrol)

In [ ]:
# 1. Kurulum
print("⏳ Kurulum...")
!pip install -q --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q unsloth_zoo
!pip install -q --no-deps xformers trl peft accelerate bitsandbytes
!pip install -q flask requests

!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
print("✅ Kurulum tamamlandı!")

In [ ]:
# 2. Google Drive
from google.colab import drive
drive.mount('/content/drive')

MODEL_PATH = "/content/drive/MyDrive/lgs_soru_tahmin_projesi/models/lgs_qwen_32b_v10"

import os
if os.path.exists(MODEL_PATH):
    print(f"✅ Model bulundu: {MODEL_PATH}")
else:
    print(f"❌ Model bulunamadı! Önce eğitimi tamamlayın.")

In [ ]:
# 3. Model Yükleme
from unsloth import FastLanguageModel
import torch

print("⏳ Model yükleniyor...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_PATH,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)
print(f"✅ Model yüklendi! VRAM: {torch.cuda.memory_allocated()/1024**3:.1f} GB")

In [ ]:
# 4. SERT System Prompt V11 (Tüm Problemler İçin)
SYSTEM_PROMPT = """Sen MEB LGS Türkçe soru yazarısın.

⛔ MUTLAK YASAKLAR (İHLAL ETTİĞİNDE SIFIR PUAN!):
1. UYDURMA kavram/terim KULLANMA! Sadece GERÇEK, BİLİNEN bilgiler!
2. Aynı cümleyi TEKRAR etme, döngüye GİRME!
3. JSON dışında TEK KARAKTER yazma!

✅ METİN FORMATI KURALLARI:
- "Ana Düşünce", "Başlık Bulma", "Anlatım Biçimi" → PARAGRAF (80-120 kelime)
- "Çok Anlamlılık", "Deyim", "Fiilimsiler", "Koşul" → 4 KISA CÜMLE (I, II, III, IV)

✅ HEDEF KELİME GÖSTERİMİ:
- Çok Anlamlılık: Kelime TİRNAK içinde → "göz", "el"
- Deyim: Deyim TİRNAK içinde → "göz kulak olmak"
- Fiilimsiler: Fiilimsi TİRNAK içinde → "koşan"

✅ ŞIK KURALLARI:
- 4 şık (A, B, C, D), benzer uzunluk
- dogru_cevap SADECE: A, B, C veya D
- UYDURMA şık YASAK!

JSON:
{"metin": "...", "soru": "...", "sik_a": "...", "sik_b": "...", "sik_c": "...", "sik_d": "...", "dogru_cevap": "A"}"""

print("✅ Sert System Prompt V11 hazır")

In [ ]:
# 5. API Servisi (V11 - OPTİMİZE PARAMETRELER)
from flask import Flask, request, jsonify
import threading
import time
import subprocess
import re

app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    prompt = data.get('prompt', '')
    
    if isinstance(prompt, dict):
        user_content = prompt.get("user", "")
    else:
        user_content = prompt

    if not user_content:
        return jsonify({'error': 'No prompt'}), 400
    
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content}
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")
    
    input_length = inputs.shape[-1]
    attention_mask = torch.ones_like(inputs)
    
    # V11 OPTİMİZE PARAMETRELER - Tüm problemler için
    outputs = model.generate(
        inputs,
        attention_mask=attention_mask,
        max_new_tokens=600,           # Daha kısa, döngü önleme
        temperature=0.35,             # Halüsinasyon önleme
        do_sample=True,
        top_p=0.8,                    # Daha odaklı
        top_k=30,                     # Daha sınırlı seçim
        repetition_penalty=1.5,       # Güçlü tekrar cezası
        no_repeat_ngram_size=4,       # 4-gram tekrarını engelle
        pad_token_id=tokenizer.eos_token_id,
    )
    
    # Sadece yeni tokenları decode et
    new_tokens = outputs[0][input_length:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)
    
    return jsonify({'result': response})

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'ok', 'model': 'LGS V11 Problem Fix'})

threading.Thread(target=app.run, kwargs={'port': 5000}).start()
print("✅ Flask API V11 başlatıldı")
print("   temp=0.35, rep_penalty=1.5, no_repeat=4")

In [ ]:
# 6. Cloudflare Tunnel
print("🚀 Tunnel açılıyor...")

process = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://127.0.0.1:5000'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(4)

for _ in range(50):
    line = process.stderr.readline().decode('utf-8')
    if not line: break
    match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
    if match:
        url = match.group(0)
        print("="*60)
        print(f"🔗 API URL: {url}")
        print("="*60)
        print(f"\n.env dosyasına ekleyin:\nCOLAB_API_URL={url}")
        break

In [ ]:
# 7. Test - Çok Anlamlılık (4 cümle formatı kontrolü)
import requests
import json

test_prompt = """Konu: Sözcükte Anlam
Alt Konu: Çok Anlamlılık

## ÇOK ANLAMLILIK KILAVUZU

**METİN FORMATI:** 4 BAĞIMSIZ CÜMLE (I, II, III, IV ile numaralandırılmış)
**HEDEF KELİME:** Her cümlede TİRNAK içinde: "göz", "el", "baş"

⛔ YASAKLAR:
- Paragraf formatı KULLANMA!
- UYDURMA kavram KULLANMA!

SADECE JSON döndür!"""

response = requests.post(
    "http://127.0.0.1:5000/generate",
    json={"prompt": {"user": test_prompt}}
)

if response.status_code == 200:
    result = response.json().get("result", "")
    print("✅ Test başarılı!")
    print("\n" + "="*60)
    print(result)
    print("="*60)
    
    # Format kontrolü
    if "I." in result or "I)" in result:
        print("\n✅ 4 CÜMLE FORMATI DOĞRU!")
    else:
        print("\n⚠️ 4 cümle formatı bulunamadı")
else:
    print(f"❌ Hata: {response.status_code}")